# MELD
Ensembling:
* use create_ensemble.py to create ensemble model. TODO: use Ensemble module + save (first part of notebook)
* put load_ensemble_model in Experiment class, use this to load model when “all” in path, or checkpoint path “ensemble_model.pt”. 
* use cross_val_aucs /  evaluate_single_model.py to create predictions

Scripts to adapt:
- ensemble.sh -> rename to evaluate_ensemble / evaluate_model? -> -> renamed to ensemble_tmp.sh can be replaced with scripts 
- ensemble_bs.sh -> use single fold predictions and have subsequenct script that calculates this -> so should probably stay as is
- adapt cross_vall_aucs to use full fold?

Scripts I’m not sure about:
* comparing_exps?
* run_evaluation_models_valsdata?

In [1]:
import meld_graph.experiment
import os
import numpy as np
import h5py
import matplotlib_surface_plotting as msp
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import nibabel as nb
from meld_classifier.paths import BASE_PATH
from meld_classifier.meld_cohort import MeldCohort,MeldSubject

from meld_graph.experiment import Experiment
from meld_graph.dataset import GraphDataset
import torch
import torch_geometric


Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1
Setting MELD_DATA_PATH to /home/co-spit1/meld_data
Setting BASE_PATH to /home/co-spit1/meld_data
Setting EXPERIMENT_PATH to /home/co-spit1/meld_experiments/co-spit1
Setting FS_SUBJECTS_PATH to /home/co-spit1/meld_data/output/fs_outputs


In [2]:
model_path1 = '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350/23-02-09_MYCZ_baseline/s_2/'

cohort = MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6.hdf5',
               dataset='MELD_dataset_V6.csv')

In [3]:
folds = np.arange(5)

exp_dirs = [os.path.join(model_path1,f'fold_0{fold}') for fold in folds]

In [4]:
exps = [Experiment.from_folder(exp_dir) for exp_dir in exp_dirs]

Initialised Experiment 23-02-09_MYCZ_baseline/s_2
Initialised Experiment 23-02-09_MYCZ_baseline/s_2
Initialised Experiment 23-02-09_MYCZ_baseline/s_2
Initialised Experiment 23-02-09_MYCZ_baseline/s_2
Initialised Experiment 23-02-09_MYCZ_baseline/s_2


In [5]:
for exp in exps:
    exp.load_model(os.path.join(exp.experiment_path, exp.experiment_name, f'fold_0{exp.fold}', 'best_model.pt'))

Creating model
Creating model
Creating model
Creating model
Creating model


In [6]:
train_ids, val_ids, test_ids = exp.get_train_val_test_ids()
dataset = GraphDataset(val_ids[:10], cohort, exp.data_parameters, mode='test')

Loading and preprocessing test data
Z-scoring data for MELD_H17_3T_FCD_0035
Z-scoring data for MELD_H21_15T_FCD_0039
Z-scoring data for MELD_H2_3T_C_0024
Z-scoring data for MELD_H19_3T_C_027
Z-scoring data for MELD_H15_3T_C_0028
Z-scoring data for MELD_H23_15T_FCD_0007
Z-scoring data for MELD_H2_3T_C_0015
Z-scoring data for MELD_H10_3T_C_0010
Z-scoring data for MELD_H19_3T_C_022
Z-scoring data for MELD_H14_3T_FCD_0019


In [7]:
data_loader = torch_geometric.loader.DataLoader(
        dataset,
        shuffle=False,
        batch_size=1)

In [8]:
from torch import nn

In [9]:
class Ensemble(nn.Module):
    """
    Ensemble models
    """
    def __init__(self, models):
        super(Ensemble, self).__init__()
        self.models = nn.ModuleList(models)
        
    def forward(self, x):
        """
        Forward pass
        """
        estimates = [model(x) for model in self.models]
        ensembled_estimates = {}
        for key in ['log_softmax', 'hemi_log_softmax', 'non_lesion_logits']:
            if key not in estimates[0].keys():
                continue
            if 'log_softmax' in key:
                # there are the logged outputs -> before mean, need to do exp 
                vals = torch.stack([torch.exp(est[key]) for est in estimates], dim=2)
                mean_val = torch.log(torch.mean(vals, dim=2))
            else:
                mean_val = torch.mean(torch.stack([est[key] for est in estimates], dim=2), dim=2)
            ensembled_estimates[key] = mean_val
        return ensembled_estimates

ensemble = Ensemble([exp.model for exp in exps])

In [10]:
# predict
for i, data in enumerate(data_loader):
    estimates = ensemble(data.x)
    
    # single folds for comparison
    #estimates_folds = [exp.model(data.x) for exp in exps]
    break

In [11]:
print(estimates['log_softmax'])

estimates_exp = [torch.exp(f['log_softmax']) for f in estimates_folds]
print(torch.log(torch.mean(torch.stack(estimates_exp, dim=2), dim=2)))

tensor([[-0.7415, -0.6470],
        [-0.6825, -0.7039],
        [-0.6615, -0.7258],
        ...,
        [-0.6694, -0.7174],
        [-0.6647, -0.7224],
        [-0.6719, -0.7149]], grad_fn=<LogBackward0>)


NameError: name 'estimates_folds' is not defined

In [14]:
print(estimates['non_lesion_logits'])
print(estimates_folds[1]['non_lesion_logits'])

tensor([[0.0771],
        [0.1123],
        [0.6275],
        ...,
        [0.2871],
        [0.3094],
        [0.3253]], grad_fn=<MeanBackward1>)
tensor([[-0.0008],
        [-0.0036],
        [ 0.0858],
        ...,
        [ 0.5935],
        [ 0.3564],
        [ 0.5201]], grad_fn=<ViewBackward0>)


In [12]:
# try saving 
fname = 'ensemble_model.pt'
torch.save(ensemble.state_dict(), fname)

In [13]:
# load model - function of experiment
import copy
def load_ensemble_model(self, checkpoint_path=None, force=False):
    if self.model is not None and not force:
        self.log.info("Model already exists. Specify force=True to force reloading and initialisation")
    # create model without checkpoint
    self.load_model()
    self.log.info('Creating ensemble model')
    models = [copy.deepcopy(self.model) for _ in range(5)]  # TODO this assumes that we are always ensembling 5 models
    ensemble_model = Ensemble(models)
    self.model = ensemble_model
    # load weights from checkpoint    
    if checkpoint_path is not None and os.path.isfile(checkpoint_path):
        # checkpoint contains both model architecture + weights
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.log.info(f"Loading ensemble model weights from checkpoint {checkpoint_path}")
        self.model.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=False)
        self.model.eval()
    


In [14]:
exp = Experiment.from_folder(exp_dirs[0])
load_ensemble_model(exp, checkpoint_path='ensemble_model.pt')

Initialised Experiment 23-02-09_MYCZ_baseline/s_2
Creating model
Creating ensemble model
Loading ensemble model weights from checkpoint ensemble_model.pt


In [15]:
# check that have different weights in different models
for name, param in exp.model.named_parameters():
    if 'encoder_conv_layers.0.0.layer.weight' in name:
        print(name, param)

models.0.encoder_conv_layers.0.0.layer.weight Parameter containing:
tensor([[-0.0221,  0.0101,  0.0660,  ..., -0.0428, -0.1122,  0.1257],
        [ 0.0143, -0.1232, -0.1224,  ...,  0.0849, -0.1058, -0.1137],
        [-0.0957, -0.0019, -0.0765,  ..., -0.1396,  0.0537, -0.0704],
        ...,
        [ 0.0984,  0.1123, -0.0475,  ..., -0.1067, -0.0964, -0.0556],
        [ 0.0423, -0.0554,  0.0813,  ..., -0.0362,  0.0256, -0.0608],
        [ 0.0188, -0.0310,  0.0262,  ..., -0.0885,  0.1057,  0.1313]],
       requires_grad=True)
models.1.encoder_conv_layers.0.0.layer.weight Parameter containing:
tensor([[ 0.0652,  0.1312,  0.0249,  ..., -0.0338, -0.0195,  0.1057],
        [-0.0112,  0.0018, -0.1078,  ..., -0.1149,  0.1087,  0.0882],
        [ 0.0339, -0.0840,  0.0183,  ..., -0.0995,  0.1258,  0.1251],
        ...,
        [-0.0856,  0.0723,  0.1273,  ...,  0.1211, -0.0441,  0.1299],
        [ 0.0139,  0.1479, -0.1146,  ..., -0.0530, -0.0415,  0.1139],
        [ 0.0209,  0.0913, -0.0453,  ...

In [16]:
# predict
for i, data in enumerate(data_loader):
    estimates2 = exp.model(data.x)
    
    break

In [17]:
(estimates['log_softmax'] == estimates2['log_softmax']).all()

tensor(True)

# test ensemble weights

In [2]:
pred_new = '/rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1/23-03-01_WRZI_classification_distance/s_0/fold_all/results/predictions.hdf5'

pred_old = ['/rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1/23-03-01_WRZI_classification_distance/s_0/fold_{:02d}/results/predictions.hdf5'.format(i) for i in range(5)]


In [3]:
exp = Experiment.from_folder(os.path.dirname(os.path.dirname(pred_new)))
exp.load_model(checkpoint_path=os.path.join(os.path.dirname(os.path.dirname(pred_new)), 'ensemble_model.pt'))

Initialised Experiment 23-03-01_WRZI_classification_distance/s_0
Creating model
Creating ensemble model
Loading ensemble model weights from checkpoint /rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1/23-03-01_WRZI_classification_distance/s_0/fold_all/ensemble_model.pt


<All keys matched successfully>
models.0.encoder_conv_layers.0.0.layer.weight Parameter containing:
tensor([[ 0.0135,  0.0678, -0.1863,  ..., -0.0700,  0.0781,  0.1409],
        [ 0.0575,  0.0788,  0.1911,  ..., -0.0310,  0.1473, -0.1491],
        [-0.0663,  0.0588,  0.0205,  ...,  0.0970,  0.0491, -0.1521],
        ...,
        [ 0.0257,  0.1267, -0.1367,  ...,  0.0021, -0.1042,  0.0584],
        [-0.0381, -0.1887, -0.0417,  ...,  0.0391, -0.0844,  0.1380],
        [-0.0173,  0.0530,  0.0020,  ...,  0.0463,  0.0730, -0.0912]],
       device='cuda:0', requires_grad=True)
models.1.encoder_conv_layers.0.0.layer.weight Parameter containing:
tensor([[ 1.7754e-02, -5.4374e-02, -1.9490e-01,  ..., -4.8245e-02,
         -1.2372e-01, -1.3056e-01],
        [-9.8838e-02,  3.7685e-02, -1.0103e-01,  ...,  4.2706e-03,
         -9.9560e-02,  1.8284e-01],
        [-1.6257e-01, -9.5077e-02,  4.9327e-02,  ..., -1.2854e-01,
         -1.5518e-01,  2.4872e-02],
        ...,
        [ 1.4830e-01, -9.6151e-0

In [4]:
# check that have different weights in different models
for name, param in exp.model.named_parameters():
    if 'encoder_conv_layers.0.0.layer.weight' in name:
        print(name, param)

models.0.encoder_conv_layers.0.0.layer.weight Parameter containing:
tensor([[ 0.0135,  0.0678, -0.1863,  ..., -0.0700,  0.0781,  0.1409],
        [ 0.0575,  0.0788,  0.1911,  ..., -0.0310,  0.1473, -0.1491],
        [-0.0663,  0.0588,  0.0205,  ...,  0.0970,  0.0491, -0.1521],
        ...,
        [ 0.0257,  0.1267, -0.1367,  ...,  0.0021, -0.1042,  0.0584],
        [-0.0381, -0.1887, -0.0417,  ...,  0.0391, -0.0844,  0.1380],
        [-0.0173,  0.0530,  0.0020,  ...,  0.0463,  0.0730, -0.0912]],
       device='cuda:0', requires_grad=True)
models.1.encoder_conv_layers.0.0.layer.weight Parameter containing:
tensor([[ 1.7754e-02, -5.4374e-02, -1.9490e-01,  ..., -4.8245e-02,
         -1.2372e-01, -1.3056e-01],
        [-9.8838e-02,  3.7685e-02, -1.0103e-01,  ...,  4.2706e-03,
         -9.9560e-02,  1.8284e-01],
        [-1.6257e-01, -9.5077e-02,  4.9327e-02,  ..., -1.2854e-01,
         -1.5518e-01,  2.4872e-02],
        ...,
        [ 1.4830e-01, -9.6151e-02,  2.1876e-01,  ...,  5.6988e-0

In [5]:
pred_old[0]

'/rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1/23-03-01_WRZI_classification_distance/s_0/fold_00/results/predictions.hdf5'

In [6]:
exp = Experiment.from_folder(os.path.dirname(os.path.dirname(pred_old[0])))
exp.load_model(checkpoint_path=os.path.join(os.path.dirname(os.path.dirname(pred_old[0])), 'best_model.pt'))

Initialised Experiment 23-03-01_WRZI_classification_distance/s_0
Creating model
Loading model weights from checkpoint /rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1/23-03-01_WRZI_classification_distance/s_0/fold_00/best_model.pt


In [7]:
# check that have different weights in different models
for name, param in exp.model.named_parameters():
    if 'encoder_conv_layers.0.0.layer.weight' in name:
        print(name, param)

encoder_conv_layers.0.0.layer.weight Parameter containing:
tensor([[ 0.0135,  0.0678, -0.1863,  ..., -0.0700,  0.0781,  0.1409],
        [ 0.0575,  0.0788,  0.1911,  ..., -0.0310,  0.1473, -0.1491],
        [-0.0663,  0.0588,  0.0205,  ...,  0.0970,  0.0491, -0.1521],
        ...,
        [ 0.0257,  0.1267, -0.1367,  ...,  0.0021, -0.1042,  0.0584],
        [-0.0381, -0.1887, -0.0417,  ...,  0.0391, -0.0844,  0.1380],
        [-0.0173,  0.0530,  0.0020,  ...,  0.0463,  0.0730, -0.0912]],
       device='cuda:0', requires_grad=True)


# test ensemble == old ensemble

In [15]:
def load_prediction(subject, hdf5, dset="prediction"):
    """load network predictions"""
    results = {}
    with h5py.File(hdf5, "r") as f:
        for hemi in ["lh", "rh"]:
            results[hemi] = f[subject][hemi][dset][:]
    return results

In [16]:
pred_new = '/rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1/23-03-01_WRZI_classification_distance/s_0/fold_all/results/predictions.hdf5'

pred_old = ['/rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1/23-03-01_WRZI_classification_distance/s_0/fold_{:02d}/results/predictions.hdf5'.format(i) for i in range(5)]


In [27]:
for subj in h5py.File(pred_new, 'r').keys():
    cur_pred_new = load_prediction(subj, pred_new)
    cur_dist_new = load_prediction(subj, pred_new, 'distance_map')
    
    cur_pred_old = [load_prediction(subj, pred_old[i]) for i in range(len(pred_old))]
    cur_dist_old = [load_prediction(subj, pred_old[i], 'distance_map') for i in range(len(pred_old))]
    cur_pred_old = {key: np.mean([el[key] for el in cur_pred_old], axis=0) for key in ['lh', 'rh']}
    cur_dist_old = {key: np.mean([el[key] for el in cur_dist_old], axis=0) for key in ['lh', 'rh']}
    
    break

In [28]:
cur_dist_new

{'lh': array([0.9781834 , 0.9768862 , 0.98737544, ..., 0.98798925, 0.98062724,
        0.9801443 ], dtype=float32),
 'rh': array([0.9708452 , 0.97811127, 0.98222953, ..., 0.9299162 , 0.9237982 ,
        0.924417  ], dtype=float32)}

In [29]:
cur_dist_old

{'lh': array([0.9782877 , 0.9769354 , 0.9873139 , ..., 0.98792744, 0.9807186 ,
        0.9801723 ], dtype=float32),
 'rh': array([0.970753  , 0.9781333 , 0.9822982 , ..., 0.9299806 , 0.92383635,
        0.92440826], dtype=float32)}

In [30]:
cur_pred_old

{'lh': array([0.00063497, 0.00057976, 0.00055508, ..., 0.00059896, 0.00059915,
        0.00059624], dtype=float32),
 'rh': array([0.00059764, 0.00055042, 0.00053709, ..., 0.0012079 , 0.00108543,
        0.00112827], dtype=float32)}

In [31]:
cur_pred_new

{'lh': array([0.00063511, 0.0005796 , 0.00055496, ..., 0.00059833, 0.00059884,
        0.00059585], dtype=float32),
 'rh': array([0.00059716, 0.0005501 , 0.00053674, ..., 0.00120773, 0.00108533,
        0.00112824], dtype=float32)}